In [27]:
from sklearn.model_selection import train_test_split
def split_data(X, y, train_size, val_size, test_size, random_state):
    #testing
    if (train_size + val_size + test_size != 1):
        raise ValueError('train_size, val_size, and test_size don\'t add up to 1')
    if not(isinstance(random_state, int)):
        raise ValueError('random state is not an integer')
    if not(isinstance(X, pd.DataFrame)):
        raise ValueError('X is not a 2D dataframe')
    if not(isinstance(y, pd.Series)):
        raise ValueError('y is not a pandas series')

    X_train, X_other, y_train, y_other = train_test_split(X,y,\
                        train_size = train_size,random_state = random_state)
    
    X_val, X_test, y_val, y_test = train_test_split(X_other,y_other,\
                        train_size = (val_size / (test_size + val_size)),random_state = random_state)

    #do more checks
    if abs(len(df.index) * train_size - len(X_train.index) ) > 1:
        raise ValueError('X_train isnt right length')
    if abs(len(df.index) * train_size - len(y_train.index)) > 1:
        raise ValueError('y_train isnt right length')
    if abs(len(df.index) * val_size - len(X_val.index)) > 1:
        raise ValueError('X_val isnt right length')
    if abs(len(df.index) * val_size - len(y_val.index)) > 1:
        raise ValueError('y_val isnt right length')
    if abs(len(df.index) * test_size - len(X_test.index)) > 1:
        raise ValueError('X_test isnt right length')
    if abs(len(df.index) * test_size - len(y_test.index)) > 1:
        raise ValueError('y_test isnt right length')

    return (X_train, y_train, X_val, y_val, X_test, y_test)

In [17]:
import pandas as pd
import numpy as np
df = pd.read_csv('new/final.csv', dtype={'citizen': 'string', 'sex': 'string', 'age': 'string', 'decision': 'string', 'geo': 'string', 'TIME_PERIOD': 'string', 'GENCONV': "Int64", 'HUMSTAT': "Int64", 'SUB_PROT': "Int64", 'REJECTED': "Int64", 'TOTAL_APPS': "Int64", "POS_RATE": "Float64"}, keep_default_na=False, na_values=['nan'])
df

,citizen,sex,age,geo,TIME_PERIOD,GENCONV,HUMSTAT,SUB_PROT,REJECTED,TOTAL_POS,TOTAL_APPS,POS_RATE
0,AD,F,UNK,AT,2008-Q1,0,0,0,0,0,0,<NA>
1,AD,F,UNK,AT,2008-Q2,0,0,0,0,0,0,<NA>
2,AD,F,UNK,AT,2008-Q3,0,0,0,0,0,0,<NA>
3,AD,F,UNK,AT,2008-Q4,0,0,0,0,0,0,<NA>
4,AD,F,UNK,AT,2009-Q1,0,0,0,0,0,0,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...
7221109,ZW,UNK,Y_LT14,UK,2019-Q3,0,0,0,0,0,0,<NA>
7221110,ZW,UNK,Y_LT14,UK,2019-Q4,0,0,0,0,0,0,<NA>
7221111,ZW,UNK,Y_LT14,UK,2020-Q1,0,0,0,0,0,0,<NA>
7221112,ZW,UNK,Y_LT14,UK,2020-Q2,0,0,0,0,0,0,<NA>


In [24]:
y = df["POS_RATE"]
X = df.drop(['GENCONV', 'HUMSTAT', 'SUB_PROT', 'REJECTED', 'TOTAL_POS', 'POS_RATE'], axis=1)
RANDOM_STATE = 14

In [28]:
X_train, y_train, X_val, y_val, X_test, y_test = split_data(X, y, 0.6, 0.2, 0.2, RANDOM_STATE)

In [36]:
times = []
for i in range(2008, 2024):
    times.append(str(i) + "-Q1")
    times.append(str(i) + "-Q2")
    times.append(str(i) + "-Q3")
    times.append(str(i) + "-Q4")

In [39]:
X_train

,citizen,sex,age,geo,TIME_PERIOD,TOTAL_APPS
105387,AF,UNK,Y_LT14,EL,2021-Q1,0
1947746,EL,UNK,Y14-17,ES,2011-Q2,0
5830790,SO,UNK,Y35-64,LU,2022-Q1,<NA>
4635724,NL,F,Y_LT14,BG,2012-Q2,<NA>
1367116,CO,M,Y14-17,LI,2008-Q3,0
...,...,...,...,...,...,...
5527879,SD,M,UNK,FR,2016-Q3,<NA>
3787158,LV,M,Y14-17,PT,2013-Q4,<NA>
7218444,ZW,UNK,Y_GE65,LV,2008-Q2,<NA>
79192,AF,F,Y_GE65,BE,2020-Q3,<NA>


In [40]:
#****************************************************preprocessing****************************************************

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

ordinal_ftrs = ['age', 'TIME_PERIOD']
ordinal_cats = [['UNK','Y_LT14','Y14-17','Y18-34','Y35-64','Y_GE65'], times]
onehot_ftrs = ['citizen', 'geo', 'sex']
minmax_ftrs = []
std_ftrs = []

# collect all the encoders
preprocessor = ColumnTransformer(
    transformers=[
        ('ord', OrdinalEncoder(categories = ordinal_cats), ordinal_ftrs),
        ('onehot', OneHotEncoder(sparse=False,handle_unknown='ignore'), onehot_ftrs),
        ('minmax', MinMaxScaler(), minmax_ftrs),
        ('std', StandardScaler(), std_ftrs)])

clf = Pipeline(steps=[('preprocessor', preprocessor)]) # for now we only preprocess 
                                                       # later on we will add other steps here

X_train_prep = clf.fit_transform(X_train)
X_val_prep = clf.transform(X_val)
X_test_prep = clf.transform(X_test)

print(X_train.shape)
print(X_train_prep.shape)
print(X_train_prep)

/Users/alexanderportland/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(4332668, 6)
(4332668, 241)
[[ 1. 52.  0. ...  0.  0.  1.]
 [ 2. 13.  0. ...  0.  0.  1.]
 [ 4. 56.  0. ...  0.  0.  1.]
 ...
 [ 5.  1.  0. ...  0.  0.  1.]
 [ 5. 50.  0. ...  1.  0.  0.]
 [ 2. 58.  0. ...  0.  0.  1.]]


In [43]:
X_train

,citizen,sex,age,geo,TIME_PERIOD,TOTAL_APPS
105387,AF,UNK,Y_LT14,EL,2021-Q1,0
1947746,EL,UNK,Y14-17,ES,2011-Q2,0
5830790,SO,UNK,Y35-64,LU,2022-Q1,<NA>
4635724,NL,F,Y_LT14,BG,2012-Q2,<NA>
1367116,CO,M,Y14-17,LI,2008-Q3,0
...,...,...,...,...,...,...
5527879,SD,M,UNK,FR,2016-Q3,<NA>
3787158,LV,M,Y14-17,PT,2013-Q4,<NA>
7218444,ZW,UNK,Y_GE65,LV,2008-Q2,<NA>
79192,AF,F,Y_GE65,BE,2020-Q3,<NA>


array([[ 1., 52.,  0., ...,  0.,  0.,  1.],
       [ 2., 13.,  0., ...,  0.,  0.,  1.],
       [ 4., 56.,  0., ...,  0.,  0.,  1.],
       ...,
       [ 5.,  1.,  0., ...,  0.,  0.,  1.],
       [ 5., 50.,  0., ...,  1.,  0.,  0.],
       [ 2., 58.,  0., ...,  0.,  0.,  1.]])